<a href="https://colab.research.google.com/github/GustavoTDini/Tech-Challenge/blob/main/FIAP_Amazon_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning do Banco de Dados da Amazon

Faremos um fine tuning para retornar a descrição de um produto com base no banco de dados da amazon

Inicialmente vamos Conectar o notebook com o Google Drive, pois como o arquivo e muito grande, temos que te-lo salvo em algum lugar de fácilo acesso.

In [ ]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Agora vamos instalar as dependencias necessárias

In [ ]:
# Vamos atualizar inicialmente o pip para podermos instalar as dependecias com segurança
!pip install --upgrade pip

In [ ]:
!pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

In [ ]:
!pip install --no-deps torch xformers trl peft accelerate bitsandbytes triton

In [ ]:
!pip install unsloth_zoo

## Parte 1 - Preparação do dataset

Agora vamos preparar nossa base de dados, vamos pegar o arquivo do database e trata-lo e transformar em um json pronto para ser lido pelo Python

In [ ]:
import json

json_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/trn.json"

def get_inicial_data(file_path):
    # Iremos iniciar abrindo duas novas coleção no python, uma para os titulos e outra para as descrições
    title_data = []
    content_data = []
    with open(file_path, 'r') as file:
        # Agora para cada linha testamos o Json se é valido
        for line in file:
            try:
                json_object = json.loads(line)
                title = json_object["title"]
                content = json_object["content"]
                # iremos eliminar os casos duplicados
                if (title != '' and content != ''):
                    title_data.append(title)
                    content_data.append(content)
            except json.JSONDecodeError:
                # e Caso o Json seja invalido ele não será incluido
                print(f"Eliminando linha com Json invalido: {line.strip()}")
                continue

    return title_data, content_data

# salvamos essas 2 listas em variaveis python
title_list, content_list = get_inicial_data(json_path)


In [ ]:
# Checaremos quantas linhas há em cada lista e se são equivalentes
print(len(title_list))
print(len(content_list))

# Vamos checar mostrando as 10 primeiras lindas de cada Json
for i in range(1,10):
    print(title_list[i])
    print(content_list[i])

In [ ]:
# Agora vamos salvar os dados em um outro Json, para evitar ter que tratar os dados novamente caso precisemos reiniciar o treinamento
json_titles_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/titles.json"
json_contents_path = "/content/drive/MyDrive/FIAP/IA/fine_tuning/contents.json"

with open(json_titles_path, 'w') as json_file:
    json.dump(title_list, json_file)

with open(json_contents_path, 'w') as json_file:
    json.dump(content_list, json_file)

E com a biblioteca pandas, vamos transformar essas 2 listas em um dataframe

In [ ]:
import pandas as pd

with open(json_titles_path, 'r') as json_file:
    data_titles = json.load(json_file)

with open(json_contents_path, 'r') as json_file:
    data_contents = json.load(json_file)

# Agora com o pandas, vamos transformar as duas listas de Json em um dataframe
data = {'title': data_titles, 'content': data_contents}
df = pd.DataFrame(data)

display(df.head())

Para finalmente chegarmos a um dataset que pode ser utilizado para o fine-tuning

In [ ]:
import datasets
from datasets import Dataset

dataset = pd.DataFrame(df)
dataset = Dataset.from_pandas(dataset)

In [ ]:
dataset

## Parte 2 - Carregar o foundation model

Agora iremos carregar o modelo que será a base do nosso fine-tuning atraves do unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

# Carregaremos o modelo inicial atraves do unsloth, no nosso caso iremos usar o modelo Llama de 8 bilhões de parametros
checkpoint_modelo = 'unsloth/Meta-Llama-3.1-8B'

In [ ]:
# Carregando o modelo e o tokenizador pré-treinados com parametros equilibrados entre tempo de computação e eficiencia
# Colocamos a sequencia maxima de tokens de 4096 para uma descrição robusta
# Dtype sendo none a detecção é automatica
# e vamos carregar em 4bits para otimizar a memória
modelo, tokenizador = FastLanguageModel.from_pretrained(
    model_name = checkpoint_modelo,
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

In [ ]:
modelo

In [ ]:
tokenizador

## Parte 3 - Testando o modelo Inicial

Agora com o modelo carregado, iremos testá-lo com um prompt

In [ ]:
prompt = "Please, describe Lord of the Rings"

prompt_tokenizado = tokenizador([prompt], return_tensors = "pt").to("cuda")

In [ ]:
prompt_tokenizado

In [ ]:
from transformers import TextStreamer

In [ ]:
FastLanguageModel.for_inference(modelo)

for i in range(10):
    streamer_texto = TextStreamer(tokenizador)
    _ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 128)

## Parte 4 - Gerando o prompt de teste

Agora iremos criar um template de prompt para termos melhores resultados - vamos utilizar a padronização do unsloth

In [ ]:
prompt_instruction = 'You are an AI assistant that helps people find information. Based on the title, or name of the product, you will generate a description of the product.'

def prompt_input(input):
    return f"Please, describe {input}"


def gerar_prompt(title, content=""):
      return f"{prompt_instruction}\n\n### Input:\n{prompt_input(title)}\n\n### Response:\n{content}"


In [ ]:
# Testando o prompt com a primeira linha do nosso dataset
print(gerar_prompt(dataset[0]['title'], dataset[0]['content']))

In [ ]:
#Precisaremos de um string marcador de fim de sentença
EOS_TOKEN = tokenizador.eos_token

In [ ]:
#Agora vamos usar nosso dataset para gerar os prompts para isso criaremos ums função que crie essa
def gerar_todos_os_prompts(data):
    titles = data['title']
    contents = data['content']
    textos = []
    for title, content in zip(titles, contents):
        texto = gerar_prompt(title, content) + EOS_TOKEN
        textos.append(texto)
    return  textos

## Parte 5 - Realizando o Fine-Tuning

E Finalmente com todos os ajustes realizados vamos realizar o fine-tuning

In [ ]:
# utilizando a biblioteca FastLanguageModel - iremos utilizar a funçao
# get_peft_model para definir os parametros que serão treinados no fine-tuning
# economizando assim, tempo computacional
# PEFT significa Parameter-Efficient Fine-Tuning
modelo_treinado = FastLanguageModel.get_peft_model(
    modelo,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 10,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
# importaremos as bibliotecas para a realização do treinamento
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
# e vamos aplicar tudo no SFTTrainer para realizar o Fine-tuning
trainer = SFTTrainer(
    model = modelo_treinado,
    tokenizer = tokenizador,
    train_dataset = dataset,
    dataset_text_field = "texto",
    max_seq_length = 4096,
    dataset_num_proc = 2,
    packing = False,
    peft_config = modelo.peft_config,
    formatting_func = gerar_todos_os_prompts,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 500,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 10,
        output_dir = "outputs",
        report_to="none", # Add this line to disable wandb logging
    ),
)
trainer.train()

Com o modelo treinado - vamos fazer um teste com o mesmo prompt anterior

In [ ]:
FastLanguageModel.for_inference(modelo)
prompt_tokenizado = tokenizador(
[gerar_prompt("Lord of the Rings")], return_tensors = "pt").to("cuda")

for i in range(10):
    streamer_texto = TextStreamer(tokenizador)
    _ = modelo_treinado.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 256)

## Parte 6 - Salvando o modelo para uso

Agora com o modelo treinado e testado - vamos salvar como uma biblioteca unsloth, para podermos acessar por outros metodos, como por exemplo o LMStudio

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Agora vamos salvar o modelo localmente
modelo_treinado.save_pretrained('modelo')
tokenizador.save_pretrained('modelo')

In [ ]:
# E podemos com esse mesmo modelo - publicar no hugging face
modelo_treinado.push_to_hub('llama-3.1-8B-amazon-content')
tokenizador.push_to_hub('llama-3.1-8B-amazon-content')

In [ ]:
modelo_treinado.save_pretrained_merged("modelo_merged", tokenizador, save_method = "merged_16bit",)